In [188]:
% Purchaseable products
% product(product_info{id:id, name:name, category:category, cost:unit_price}).

% Clients
% client(client_info{name:name, phone:phone_number}).

% Purchase transactions
% purchase(purchase_info{id:purchase_id, phone:client_phone, product_id:product_id, quantity:quantity, day_number:day_number}).

% Client tiers, based on how much the client has spent
% tier(tier_name, threshold)

% This line is for interactive use.

:- dynamic product/1.
:- dynamic purchase/1.
:- dynamic client/1.


?- consult("DATABASE-ver2.plfact").

% This line adds an invocation to the main program target.
% The main program will be defined after this.
prolog :- 
    consult("DATABASE-ver2.plfact"),
    fail.

?- findall(Info, product(Info), Products).

% The Prolog server was restarted

true

Products = [product_info{category:Mолочный отдел,cost:120,id:1,name:Молоко},product_info{category:Mолочный отдел,cost:230,id:2,name:Творог},product_info{category:Mолочный отдел,cost:300,id:3,name:Сыр},product_info{category:Mолочный отдел,cost:90,id:4,name:Кефир},product_info{category:Овощи,cost:35,id:5,name:Картофель},product_info{category:Овощи,cost:12,id:6,name:Лук},product_info{category:Овощи,cost:56,id:7,name:Томаты},product_info{category:Овощи,cost:22,id:8,name:Морковь},product_info{category:Мясо,cost:21,id:9,name:Голень куриная},product_info{category:Мясо,cost:48,id:10,name:Свиная шея},product_info{category:Мясо,cost:129,id:11,name:Окорок бараний},product_info{category:Мясо,cost:43,id:12,name:Филе индейки},product_info{category:Бакалея,cost:60,id:13,name:Греча},product_info{category:Бакалея,cost:25,id:14,name:Манная крупа},product_info{category:Бакалея,cost:105,id:15,name:Овсяная крупа},product_info{category:Бакалея,cost:95,id:16,name:Рис},product_info{category:Бакалея,cost:58,id

In [189]:
% A repeat client is defined as a customer who has multiple purchases made on different days.
repeat_client(PID) :-
    purchase(P1), _{phone:PID, day_number: D1} :< P1,
    purchase(P2), _{phone:PID, day_number: D2} :< P2,
    not(D1 = D2).
%    purchase(pid, _f3, _f4, d2).
?- findall(X, repeat_client(X), _RC), list_to_set(_RC, RepeatClients).

% Asserting clauses for user:repeat_client/1


RepeatClients = [+7 999 123 45 67,+7 926 987 65 43,+7 965 432 10 98]

In [190]:
% (internal) The price of a particular purchase is the number of items involved in that purchase,
% multiplied by the cost of an item.
purchase_price(PurchaseId, PurchasePrice) :-
    purchase(P), _{id: PurchaseId, product_id: PI, quantity: Q} :< P,
    product(Pr), _{id: PI, cost: Uprice} :< Pr,
    PurchasePrice is Q*Uprice.
?- purchase_price(1, X).

% Asserting clauses for user:purchase_price/2


X = 300

In [191]:
% (internal) The list of all purchases on a particular day, along with their prices.
day_purchase_prices(Day, Plist) :- 
    findall(PurchaseId, (
        purchase(Purch),
        _{id: PurchaseId, quantity: Count, day_number: Day} :< Purch
    ), DayPurchases),  % gather list of all purchases for the day
    findall((PurchaseId, PurchasePrice), % then associate purchaseids with a purchaseprice
        (
            member(PurchaseId, DayPurchases), % for the purchaseids in the purchases of the day only
            purchase_price(PurchaseId, PurchasePrice)
        ),
        Plist).

?- day_purchase_prices(12, P).

% Asserting clauses for user:day_purchase_prices/2


P = [(1,300),(2,105),(3,24)]

In [192]:
% The cashflow of a day is defined as the total sum of the purchases made on that day.
cashflow(Day, Amount) :-
    day_purchase_prices(Day, Plist), % First gather the list of purchases with their costs
    findall(Cost, (
        member((_, Cost), Plist),  % then extract the second item from each tuple
        true
    ),AmountList),  % and gather those into a list
    sum_list(AmountList, Amount). % which is then folded

?- cashflow(12, X).

% Asserting clauses for user:cashflow/2


X = 429

In [193]:
% (internal) The list of purchases made by a specific client, along with their prices.
client_purchase_prices(Cid, Plist) :- 
    client(C), _{phone: Cid} :< C,  % There must be a client with this id.
    findall(PurchaseId, (
        purchase(Purch),
        _{id: PurchaseId, phone: Cid, quantity: Count} :< Purch
    ), ClientPurchases),
    findall((PurchaseId, PurchasePrice),
        (
            member(PurchaseId, ClientPurchases), % for the purchaseids in the purchases of the day only
            purchase_price(PurchaseId, PurchasePrice)
        ),
        Plist).

?- client_purchase_prices("+7 999 123 45 67", X).

% Asserting clauses for user:client_purchase_prices/2


X = [(1,300),(2,105),(3,24),(4,258),(5,25),(6,58),(7,300),(8,140),(9,90),(10,215),(11,120),(12,105),(13,120),(43,230),(44,300),(45,48),(46,144)]

In [194]:
% The total sum paid by the client across all their purchases.
client_spent(Cid, Amount) :-
    client(C), _{phone: Cid} :< C,  % There must be a client with this id.
    client_purchase_prices(Cid, Plist),
    findall(Cost, member((_, Cost), Plist), AmtList),
    sum_list(AmtList, Amount).
?- findall((X,Y), client_spent(X,Y), Spendings).

% Asserting clauses for user:client_spent/2


Spendings = [(+7 999 123 45 67,2582),(+7 926 987 65 43,2052),(+7 925 555 44 33,1334),(+7 965 432 10 98,4087)]

In [195]:
% The client's tier is defined as the largest tier whose threshold the client's purchase sum has exceeded.
client_tier(Cid, Tier) :-
    client(C), _{phone: Cid} :< C,  % There must be a client with this id.
    client_spent(Cid, Amount),  % Check how much the client spent
    findall(PassedTierAmt, 
    (
        tier(_, PassedTierAmt),  % Get all tier amounts
        PassedTierAmt =< Amount  % that are smaller than that
    ), PassedTierAmts),
    max_list(PassedTierAmts, MaxTierAmt),  % then find the largest one
    tier(Tier, MaxTierAmt).  % and return the tier corresponding to it
    
?- findall((X,Y), client_tier(X,Y), CTiers).

% Asserting clauses for user:client_tier/2


CTiers = [(+7 999 123 45 67,silver),(+7 926 987 65 43,silver),(+7 925 555 44 33,bronze),(+7 965 432 10 98,platinum)]

In [196]:
% The list of products per day is defined as the list of product names that feature in purchases for that day
products_per_day(Day, ProductList) :- 
    findall(ProductId, (
        purchase(Purch), _{product_id: ProductId, day_number: Day} :< Purch
    ), ProductIdList),  % gather IDs
    findall(ProductName, (
        product(Prod), _{name: ProductName, id: Id} :< Prod, % then convert ID to name
        member(Id, ProductIdList)
    ), ProductList).
?- products_per_day(13, X).

% Asserting clauses for user:products_per_day/2


X = [Филе индейки,Манная крупа,Сахар]

In [197]:
% The list of clients per product is defined as the list of names of the clients for whom there exists a purchase
% for the product with the given ID.
buyers_of_product(Pid, Buyers) :- 
    findall(BuyerId, (
        purchase(Purch), _{product_id: Pid, phone: BuyerId} :< Purch
    ), BuyerIds),  % gather list of buyer IDs
    list_to_set(BuyerIds, UniqBuyerIds),  % uniquify
    findall(Name, (
        client(C), _{phone: Bid, name: Name} :< C, % to names
        member(Bid, UniqBuyerIds)
    ), Buyers).

?- findall((1, Names), buyers_of_product(1, Names), Answers).
?- findall((20, Names), buyers_of_product(20, Names), Answers).

% Asserting clauses for user:buyers_of_product/2


Answers = [(1,[Анна Иванова,Илья Петров,Сергей Кузнецов])]

Answers = [(20,[Анна Иванова,Сергей Кузнецов])]

In [198]:
% The list of categories by client is defined as the list of product categories that this client has bought.
categories_by_client(Cid, Cats) :-
    findall(ProductId, (
        purchase(Purch), _{phone: Cid, product_id: ProductId} :< Purch
    ), ProductIds), % Gather list of categories
    findall(Cat, (
        member(Pid, ProductIds),  % for every product
        product(Prod), _{id: Pid, category: Cat} :< Prod   % find its category
    ), CatsMultiple),
    list_to_set(CatsMultiple, Cats).
?- findall((Cid, Cats), (client(C), _{phone: Cid} :< C, categories_by_client(Cid, Cats)), Answers).

% Asserting clauses for user:categories_by_client/2


Answers = [(+7 999 123 45 67,[Mолочный отдел,Овощи,Мясо,Бакалея]),(+7 926 987 65 43,[Mолочный отдел,Овощи,Мясо,Бакалея]),(+7 925 555 44 33,[Mолочный отдел,Овощи,Мясо,Бакалея]),(+7 965 432 10 98,[Mолочный отдел,Овощи,Бакалея])]

In [199]:
% A product category is defined as a (unique) category that a product has.

product_cats_internal(Cats) :- findall(Cat, (product(Prod), _{category: Cat} :< Prod), CatsMulti), sort(CatsMulti, Cats).
product_category(Cat) :- product_cats_internal(Cats), member(Cat, Cats).

?- findall(X, product_category(X), Xs).

% Asserting clauses for user:product_cats_internal/1


% Asserting clauses for user:product_category/1


Xs = [Mолочный отдел,Бакалея,Мясо,Овощи]

In [200]:
% ------ Examples ------
example1:-
    writeln("---- Example 1 ----"),
    writeln("Get list of repeat clients:"),
    setof(RepC, repeat_client(RepC), RepeatClients),
    writeln(RepeatClients).

% Asserting clauses for user:example1/0


In [201]:
example2 :-
    writeln("---- Example 2 ----"),
    writeln("Get each client's total spending and tier:"),
    (
        client(C), _{name: Name, phone: Id} :< C,
        write(Name), write(":"),
        client_spent(Id, Spent),
        write(Spent), write(":"),
        client_tier(Id, Tier),
        write(Tier),
        nl,
        fail()
    );
    true.

% Asserting clauses for user:example2/0


In [202]:
example3 :-
    writeln("---- Example 3 ----"),
    writeln("Get the products purchased on every day:"),
    findall(Day,(purchase(Purch), _{day_number: Day} :< Purch), DaysMulti),
    list_to_set(DaysMulti, Days),
    (
        member(Day, Days),
        products_per_day(Day, ProductsMulti),
        list_to_set(ProductsMulti, Products),
        write(Day), write(":"), write(Products), nl, fail()
    );
    true.

% Asserting clauses for user:example3/0


In [203]:
example4 :-
    writeln("---- Example 4 ----"),
    writeln("Get the cashflow for every day:"),
    findall(Day,(purchase(Purch), _{day_number: Day} :< Purch), DaysMulti),
    list_to_set(DaysMulti, Days),
    (
        member(Day, Days),
        cashflow(Day, Cash),
        write(Day), write(":"), write(Cash), nl, fail()
    );
    true.

% Asserting clauses for user:example4/0


In [204]:
example5 :-
    writeln("---- Example 5 ----"),
    writeln("Get the categories of product purchased by every client:"),
    writeln("(the last client does not purchase meat products)"),
    (
        client(C), _{name: ClientName, phone: Cid} :< C,
        categories_by_client(Cid, Cats),
        write(ClientName), write(":"), write(Cats), nl, fail()
    );
    true.

% Asserting clauses for user:example5/0


In [205]:
example6 :-
    writeln("---- Example 6 ----"),
    writeln("Which categories does a client not buy?"), fail;
    (
        product_category(Category), % Get a product category,
        client(C), _{name: ClientName, phone: Cid} :< C,  % Then get a client,
        \+ (  % And make sure that
            purchase(Purch), _{product_id: Pid, phone: Cid} :< Purch, % there is no purchase such
            product(Prod), _{id: Pid, category: Category} :< Prod % that the product has the given category,
        ),
        write("Client "), write(ClientName), write(" does not buy category "), write(Category), nl,
        fail
    );
    true.

% Asserting clauses for user:example6/0


In [206]:
say_hi(Request) :- 
    reply_json(_{
        hello: world,
        try_me: [
            _{method: "GET", url:"/purchases"},
            _{method: "GET", url:"/clients"},
            _{method: "GET", url:"/products"},
            _{method: "GET", url:"/products/<:id>", args: _{id: int}},
            _{method: "POST", url:"/products", data: _{id: int, name: str, category: str, cost: int}},
            _{method: "POST", url:"/purchases", data: _{id: int, phone: str, product_id: int, quantity: int, day: int}},
            _{method: "POST", url:"/clients", data: _{phone: str, name: str}}

        ]
    }).

product_info(Pid, Prod) :-
    product(Prod), _{id: Pid} :< Prod.


products_list(Request) :- 
    findall(Prod, product(Prod), Products),
    reply_json(Products).

product(get, ProductIdStr, Request) :-
    atom_number(ProductIdStr, ProductId),
    product_info(ProductId, Prod),
    reply_json(Prod).

purchase_info(Pid, Purch) :- 
    purchase(Purchinfo), _{id: Pid, phone: Cid, product_id: ProdId, quantity: Quantity, day_number: Day} :< Purchinfo,
    client_info(Cid, Cli),
    product_info(ProdId, Prod),
    Purch = _{id: Pid, client_id: Cid, product_id: ProdId, quantity: Quantity, day: Day,
        expand: _{
            client: Cli,
            product: Prod
        }%,
    }.

purchases_list(Request) :-
    findall(Purch, purchase_info(Pid, Purch), Purchases),
    reply_json(Purchases).

purchase(get, PurchaseIdStr, Request) :-
    atom_number(PurchaseIdStr, PurchaseId),
    purchase_info(PurchaseId, Purch),
    reply_json(Purch).

client_info(Cid, Cli) :- 
    client(_{name:Name, phone:Cid}),  % As an example, some properties of the client are being calculated here
    client_spent(Cid, Amt),
    client_tier(Cid, Tier),
    Cli = _{name: Name, id: Cid, amount_spent: Amt, loyalty_tier: Tier}.

clients_list(Request) :-
    findall(Cli, client_info(Cid, Cli), Clients),
    reply_json(Clients).

client(get, ClientIdStr, Request) :-
    atom_number(ClientIdStr, ClientId),
    client_info(ClientId, Client),
    reply_json(Client).
    
    

product(post, Request) :-
    http_read_json(Request, NewProduct, [json_object(dict)]),
    _{id: IdStr, name: Name, category: Cat, cost: CostStr} :< NewProduct,
    atom_number(IdStr, IdInt), atom_number(CostStr, CostInt),
    assert(product(product_info{id: IdInt, name: Name, category: Cat, cost: CostInt})),
    
    format(string(Ref), "/products/~d", [IdInt]),
    reply_json(_{status: ok, ref: Ref}).

client(post, Request) :- 
    http_read_json(Request, NewClient, [json_object(dict)]),
    _{name: Name, phone: Phone} :< NewClient,
    assert(client(client_info{name: Name, phone: Phone})),
    reply_json(_{status: ok}).


purchase(post, Request) :-
    http_read_json(Request, NewPurchase, [json_object(dict)]),
    _{
        id: IdStr,
        phone: Phone,
        quantity: QuantityStr,
        day: DayNumberStr
    } :< NewPurchase,
    atom_number(IdStr, Id),
    atom_number(ProductIdStr, ProductId),
    atom_number(QuantityStr, Quantity),
    atom_number(DayNumberStr, DayNumber),
    assert(purchase(purchase_info{
        id: Id, phone: Phone,
        product_id: ProductId,
        quantity: Quantity,
        day: DayNumber
    })),
    format(string(Ref), "/purchases/~d", [Id]),
    reply_json(_{status: ok, ref: Ref}).

products_root(get, Request) :- products_list(Request).
products_root(post, Request) :- product(post, Request).

purchases_root(get, Request) :- purchases_list(Request).
purchases_root(post, Request) :- purchase(post, Request).


    

example7 :- 
    writeln("---- Example 7 ----"),
    writeln("Run a simple HTTP server that returns JSON data about the database"),
    writeln("and allows adding new facts into the database (in memory only)"),
    fail;
    
% If the `run_web_server/0` is not defined, then exit early with a message.
    \+ current_predicate(run_web_server/0),
    writeln("!!!!! The web server is disabled in the config. Assert `run_web_server/0` to enable it"),
    true;
    
% If it is defined and true, instantiate the server.
    current_predicate(run_web_server/0),
    run_web_server,
    use_module(library(http/thread_httpd)),
    use_module(library(http/http_dispatch)),
    use_module(library(http/json_convert)),
    use_module(library(http/http_json)),
    http_handler(root(.), say_hi, []),
    http_handler(root(products), products_root(Method), [methods([get, post])]),
    http_handler(root(products/ProductId), product(get, ProductId), [methods([get])]),
    http_handler(root(purchases), purchases_root(Method), [methods([get, post])]),
    http_handler(root(purchases/PurchaseId), purchase(Methos, PurchaseId), [methods([get])]),
    http_handler(root(clients), clients_root(Method), [methods([get, post])]),

    
    
    
    writeln("Starting server on port 8080!!"),
    http_server(http_dispatch, [port(8080)]),
    writeln("Type any Prolog term then press Enter to stop..."),
    read(X).

% Asserting clauses for user:say_hi/1


% Asserting clauses for user:product_info/2


% Asserting clauses for user:products_list/1


% Asserting clauses for user:product/3


% Asserting clauses for user:purchase_info/2


...

% Asserting clauses for user:client/2


% Asserting clauses for user:purchase/2


% Asserting clauses for user:products_root/2


% Asserting clauses for user:purchases_root/2


% Asserting clauses for user:example7/0


In [ ]:
prolog :-
    example1,
    example2,
    example3,
    example4,
    example5,
    example6,
    example7.
?- prolog.